In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from  PIL import Image
import numpy as np
from tensorflow.keras import Input, Model
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import split_dataset
import tensorflow as tf
from tensorflow import keras


from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split




#dataset_path = '../data/images/'
RSEED=42

In [ ]:
# copy the contents of images folder, excluding Cassava, Rice, Potato and Sugercane

import os
import shutil

# Source and destination folder paths
source_folder = '../data/images_data'
destination_folder = '../data/images'

# Create destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# List of prefixes to exclude
excluded_prefixes = ['Cassava', 'Rice', 'Potato', 'Sugercane']

# Function to check if the name starts with any of the excluded prefixes
def is_excluded(name):
    for prefix in excluded_prefixes:
        if name.startswith(prefix):
            return True
    return False

# Iterate over files in the source folder
for item in os.listdir(source_folder):
    source_item = os.path.join(source_folder, item)
    destination_item = os.path.join(destination_folder, item)
    
    # Check if the item is a file
    if os.path.isfile(source_item):
        # Check if the file name starts with an excluded prefix
        if not is_excluded(item):
            # Copy the file to the destination folder
            shutil.copy(source_item, destination_item)
    # Check if the item is a folder
    elif os.path.isdir(source_item):
        # Check if the folder name starts with an excluded prefix
        if not is_excluded(item):
            # Copy the folder recursively to the destination folder
            shutil.copytree(source_item, destination_item)

print("Contents copied successfully!")


In [4]:
#import data

data = []

# Specify the path to your dataset
dataset_path = '../data/images/'

# Iterate through each plant folder
for plant_class in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, plant_class)
    
    # Iterate through each image in the plant folder
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        data.append({'Image_Path': image_path, 'Class': plant_class})

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data) #.reset_index(drop=True, inplace=True)

df

,Image_Path,Class
0,../data/images/Tomato___spider_mites/fa05ccd2-...,Tomato___spider_mites
1,../data/images/Tomato___spider_mites/fd3b6292-...,Tomato___spider_mites
2,../data/images/Tomato___spider_mites/82306885-...,Tomato___spider_mites
3,../data/images/Tomato___spider_mites/9c7f0991-...,Tomato___spider_mites
4,../data/images/Tomato___spider_mites/d7c5cad4-...,Tomato___spider_mites
...,...,...
57689,../data/images/Soybean___healthy/57c18b39-2a33...,Soybean___healthy
57690,../data/images/Soybean___healthy/4fdc663e-a8ea...,Soybean___healthy
57691,../data/images/Soybean___healthy/df807f13-078b...,Soybean___healthy
57692,../data/images/Soybean___healthy/60bf9858-951a...,Soybean___healthy


In [6]:
def rename_subfolder(parent_dir, subfolder_name, new_name):
    subfolder_path = os.path.join(parent_dir, subfolder_name)
    new_path = os.path.join(parent_dir, new_name)

    # Check if the subfolder exists
    if os.path.exists(subfolder_path):
        # Rename the subfolder
        os.rename(subfolder_path, new_path)
        print(f"{subfolder_name} renamed to {new_name} successfully.")
    else:
        print(f"Subfolder {subfolder_name} doesn't exist in {parent_dir}.")

# Example usage
parent_directory = "../data/images/"
subfolder_name = "Grape_leaf_blight"
new_name = "Grape___leaf_blight"

rename_subfolder(parent_directory, subfolder_name, new_name)

Subfolder Grape_leaf_blight doesn't exist in ../data/images/.


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57694 entries, 0 to 57693
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Image_Path  57694 non-null  object
 1   Class       57694 non-null  object
dtypes: object(2)
memory usage: 901.6+ KB


In [8]:
# Extract plant types from class names
df['plant_type'] = df['Class'].apply(lambda x: x.split('___')[0])

# Count the occurrences of each plant type
plant_type_counts = df['plant_type'].value_counts()

# Display the count of each plant type
print(plant_type_counts)
#plant_type_counts

plant_type
Tomato         18160
Apple           6542
Grape           6232
Orange          5507
Soybean         5090
Corn            3852
Peach           2657
Bell_pepper     2475
Cherry          1906
Squash          1835
Strawberry      1565
Blueberry       1502
Raspberry        371
Name: count, dtype: int64


In [9]:
problematic_rows = df[df['Class'].apply(lambda x: len(x.split('___')) < 2)]
print("Problematic Rows:")
print(problematic_rows)

Problematic Rows:
Empty DataFrame
Columns: [Image_Path, Class, plant_type]
Index: []


In [10]:

#problematic_rows = df[df['Class'].str.contains('Grape_leaf_blight')]
df['Class'] = df['Class'].replace('Grape_leaf_blight', 'Grape___leaf_blight', regex=True)


In [11]:
# Extract disease types from class names
df['disease_type'] = df['Class'].apply(lambda x: x.split('___')[1])

# Count the occurrences of each disease type
disease_type_counts = df['disease_type'].value_counts()

# Display the count of each disease type
#print(disease_type_counts)
print(disease_type_counts)

disease_type
healthy                 16739
citrus_greening          5507
bacterial_spot           5421
leaf_curl                5357
powdery_mildew           2887
black_measles            2271
late_blight              1909
black_rot                1801
septoria_leaf_spot       1771
spider_mites             1676
target_spot              1404
rust                     1241
scab                     1222
common_rust              1192
leaf_scorch              1109
isariopsis_leaf_spot     1076
early_blight             1000
northern_leaf_blight      985
leaf_mold                 952
gray_leaf_spot            513
leaf_blight               400
gray_spot                 395
mosaic_virus              373
alternaria_leaf_spot      278
brown_spot                215
Name: count, dtype: int64


In [12]:
df['disease_type'].nunique()

25

In [13]:
labels = np.unique(df['disease_type']).tolist()

labels

['alternaria_leaf_spot',
 'bacterial_spot',
 'black_measles',
 'black_rot',
 'brown_spot',
 'citrus_greening',
 'common_rust',
 'early_blight',
 'gray_leaf_spot',
 'gray_spot',
 'healthy',
 'isariopsis_leaf_spot',
 'late_blight',
 'leaf_blight',
 'leaf_curl',
 'leaf_mold',
 'leaf_scorch',
 'mosaic_virus',
 'northern_leaf_blight',
 'powdery_mildew',
 'rust',
 'scab',
 'septoria_leaf_spot',
 'spider_mites',
 'target_spot']

In [14]:
#import os
import shutil
from sklearn.model_selection import train_test_split

# Split data into features (image paths) and labels (classes)
X = df['Image_Path']
y = df['disease_type']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

# Define paths for the training and testing data folders
train_dir = '../data/train/'
test_dir = '../data/test/'

# Create directories if they don't exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Copy images to the training folder
for image_path, label in zip(X_train, y_train):
    dest_folder = os.path.join(train_dir, label)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    shutil.copy(image_path, dest_folder)

# Copy images to the testing folder
for image_path, label in zip(X_test, y_test):
    dest_folder = os.path.join(test_dir, label)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    shutil.copy(image_path, dest_folder)